### 문제 1-3 : FewShotPromptTemplate과 시스템 메시지 활용

In [4]:
from dotenv import load_dotenv
import os
from langchain_core.prompts import (
    FewShotChatMessagePromptTemplate,
    ChatPromptTemplate
)
from langchain_openai import ChatOpenAI

# .env 파일 불러와서 환경변수로 설정
load_dotenv(dotenv_path='.env')
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# 예시 프롬프트 (대화형)
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{news}"),
    ("ai", "키워드: {keywords}")
])

examples = [
    {
        "news": "삼성전자가 내년 초에 자체적으로 개발한 인공지능(AI) 가속기를 처음으로 출시할 예정이다. 이는 AI 반도체 시장에서 지배적인 위치를 차지하고 있는 엔비디아의 독점을 도전하고, 세계 최고의 반도체 제조업체로서의 지위를 다시 확립하려는 삼성전자의 노력으로 해석된다.",
        "keywords": "삼성전자, 인공지능, 엔비디아"
    },
    {
        "news": "세계보건기구(WHO)는 최근 새로운 건강 위기에 대응하기 위해 국제 협력의 중요성을 강조했다. 전염병 대응 역량의 강화와 글로벌 보건 시스템의 개선이 필요하다고 발표했다.",
        "keywords": "세계보건기구, 건강위기, 국제협력"
    },
    {
        "news": "올해 상반기 상용근로자의 월평균 임금총액 인상률이 전년 동기 대비 3.5%로 나타났다. 지난해 같은 기간 2.2%보다 1.3%포인트(p)나 확대된 것이다. 대기업 성과급 등 특별급여가 크게 증가한 영향으로 풀이된다.",
        "keywords": "상용근로자, 월평균 임금총액, 인상률"
    }
]

# Few-Shot 프롬프트
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# 최종 프롬프트
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 뉴스 키워드 추출 전문가입니다. 핵심 키워드 3개를 추출하세요."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

# 모델 생성 및 체인 구성
#llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",
    model="openai/gpt-oss-120b",
    temperature=0.7
)

chain = final_prompt | llm

# 테스트 실행
result = chain.invoke({"input": "제미나이 2.0 플래시는 현재 구글 AI 스튜디오(Google AI Studio) 및 버텍스 AI(Vertex AI)에서 제미나이 API를 통해 개발자에게 실험 모델로 제공됩니다. 모든 개발자는 멀티모달 입력 및 텍스트 출력을 사용할 수 있으며, 텍스트 음성 변환(text-to-speech) 및 네이티브 이미지 생성은 일부 파트너들을 대상으로 제공됩니다. 내년 1월에는 더 많은 모델 사이즈와 함께 일반에 공개될 예정입니다."
})
print(result.content)

result = chain.invoke({"input": "도널드 트럼프 미국 대통령이 영국을 두 번째로 국빈 방문한 가운데, 런던 도심에서는 수천 명이 모여 그의 정책과 정치적 행보에 반대하는 시위를 벌였다. 가디언 등 외신에 따르면, 트럼프 대통령은 국빈 만찬에서 미국과 영국 간의 ‘특별한 관계’를 강조하며 이번 국빈 방문이 “내 인생 최고의 영예 중 하나”라고 말했다. 그러나 같은 날 런던 곳곳에서 열린 대규모 시위는 이와 대조적 풍경을 이뤘다."
})
print(result.content)

result = chain.invoke({"input": "부산 사직야구장에는 롯데 자이언츠가 전날 대승의 기세를 이어갈지에 시선이 쏠렸다. 관중들은 롯데의 연승을 기대했지만, 4회 대량 실점 이후 분위기는 냉각됐다. 롯데는 키움 히어로즈에 5-15로 완패하며 하루 만에 다시 6위로 밀려났다."
})
print(result.content)

키워드: 제미나이 2.0 플래시, 구글 AI 스튜디오·버텍스 AI, 멀티모달 입력·출력
키워드: 도널드 트럼프, 영국 국빈 방문, 대규모 시위
키워드: 롯데 자이언츠, 키움 히어로즈, 사직야구장
